In [1]:
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import secrets

In [3]:
sql_engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{user}'.format(
    user=secrets.pg_user,
    password=secrets.pg_password,
    host='localhost',
    port='5433'
))

In [4]:
with sql_engine.connect() as connection:
    id, key, val = connection.execute(
        'select * from foo limit 1'
    ).fetchone()
(id, key, val)

(1, 'a', '1')

## Connect to remote DB via SSH

adapted from https://stackoverflow.com/questions/31506958/sqlalchemy-through-paramiko-ssh#31508516

In [18]:
from sshtunnel import SSHTunnelForwarder
from sqlalchemy.orm import sessionmaker

def with_remote_sql_session(function):
    with SSHTunnelForwarder(
            (secrets.server_ip_address, 22),
            ssh_username=secrets.ssh_username,
            ssh_pkey=secrets.ssh_private_key_path,
            ssh_private_key_password=secrets.ssh_private_key_password,
            remote_bind_address=('127.0.0.1', 5433)
        ) as server:

        server.start() #start ssh sever

        #connect to PostgreSQL
        local_port = str(server.local_bind_port)
        engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{user}'.format(
            user=secrets.pg_user,
            password=secrets.pg_password,
            host='localhost',
            port=local_port
        ))

        Session = sessionmaker(bind=engine)
        session = Session()
        try:
            return_val = function(session)
        finally:
            session.close()
            return return_val


In [19]:
def get_foo(session):
    return session.execute('select * from foo').fetchone()

with_remote_sql_session(get_foo)

(1, 'key', 'val')